In [71]:
from tesi.database.di import get_session_maker
from tesi.zappai.di import get_cds_api, get_crop_repository, get_crop_yield_data_repository, get_location_repository
from tesi.zappai.exceptions import CropNotFoundError
from tesi.zappai.repositories.climate_generative_model_repository import FEATURES as CLIMATE_GENERATIVE_MODEL_FEATURES
from tesi.zappai.repositories.dtos import CropYieldDataDTO

session_maker = get_session_maker()
location_repository = get_location_repository(session_maker=session_maker)
crop_repository = get_crop_repository(session_maker=session_maker)
crop_yield_data_repository = get_crop_yield_data_repository(
    session_maker=session_maker,
    crop_repository=crop_repository,
    location_repository=location_repository
)

crop = await crop_repository.get_crop_by_name("maize")
if crop is None:
    raise CropNotFoundError()
crop_yield_data = await crop_yield_data_repository.get_crop_yield_data(crop_id=crop.id)
crop_yield_data_df = CropYieldDataDTO.from_list_to_dataframe(crop_yield_data)
crop_yield_data_df


,index,id,location_id,crop_id,sowing_year,sowing_month,harvest_year,harvest_month,_yield
0,692,80e4fecf-e240-46b9-b144-950c38b97493,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2400.0
1,693,d97306b4-053d-4f31-b31c-ad2e1163ddf2,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2500.0
2,1022,c14f8468-8ca0-454a-9e06-c372a372efd4,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,3093.0
3,1023,5660858a-071e-4cb0-9ad5-14d808b0fb47,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,5678.0
4,1648,c92dc64d-9b4f-4a28-a787-f36913b43cec,423761b0-10b5-4ffa-b56b-ef5df7215e85,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,4,1980,10,4400.0
...,...,...,...,...,...,...,...,...,...
1648,73,9c64d4e8-6c5a-4f6b-9c99-196810c096f7,d7636c4f-6399-4275-8fab-4d3f531d898a,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,9,4050.0
1649,95,feff9efb-915a-460d-9235-fc4d3b65fc42,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,10895.0
1650,101,8c4c0099-7b29-413a-9094-2b81143d84aa,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,12114.0
1651,794,232bd556-4455-41b8-ae2e-92c1d45a758a,e22382f9-1e49-4eb5-acad-9e7d45a9e788,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,5,2017,9,9670.0


In [72]:
import pandas as pd
from tesi.zappai.di import (
    get_future_climate_data_repository,
    get_past_climate_data_repository,
)
from tesi.zappai.exceptions import LocationNotFoundError
from tesi.zappai.repositories.dtos import ClimateDataDTO

cds_api = get_cds_api()
past_climate_data_repository = get_past_climate_data_repository(
    session_maker=session_maker,
    cds_api=cds_api,
    location_repository=location_repository,
)
future_climate_data_repository = get_future_climate_data_repository(
    session_maker=session_maker, cds_api=cds_api
)

enriched_crop_yield_data_df = pd.DataFrame()

processed = 0


def print_processed():
    print(f"\rPROCESSED {processed}/{len(crop_yield_data_df)}", end="")


print_processed()

for index, row in crop_yield_data_df.iterrows():
    location = await location_repository.get_location_by_id(row["location_id"])
    if location is None:
        raise LocationNotFoundError()
    past_climate_data_df = ClimateDataDTO.from_list_to_dataframe(
        await past_climate_data_repository.get_past_climate_data(
            location_id=location.id,
            year_from=row["sowing_year"],
            month_from=row["sowing_month"],
            year_to=row["harvest_year"],
            month_to=row["harvest_month"],
        )
    )
    past_climate_data_df = past_climate_data_df[CLIMATE_GENERATIVE_MODEL_FEATURES]
    stats = ["mean", "sum", "std", "min", "max"]
    climate_data_stats = past_climate_data_df.agg(
        {feature: stats for feature in CLIMATE_GENERATIVE_MODEL_FEATURES},
        axis=0,
    )
    result_climate_data_stats_df = pd.DataFrame()
    for feature in CLIMATE_GENERATIVE_MODEL_FEATURES:
        for stat in stats:
            result_climate_data_stats_df[f"{feature}_{stat}"] = [
                climate_data_stats.loc[stat][feature]
            ]
    processed += 1
    # convert the row to a DataFrame
    crop_yield_data_row_df = pd.DataFrame([row])
    # since the row was a Series, remove the useless index column that the DataFrame inherited
    crop_yield_data_row_df = crop_yield_data_row_df.drop(columns=["index"])
    crop_yield_data_row_df = crop_yield_data_row_df.reset_index(drop=True)
    enriched_crop_yield_data_row = pd.concat([crop_yield_data_row_df, result_climate_data_stats_df], axis=1)
    enriched_crop_yield_data_df = pd.concat(
        [
            enriched_crop_yield_data_df,
            enriched_crop_yield_data_row
        ],
        axis=0,
    )
    print_processed()

enriched_crop_yield_data_df = enriched_crop_yield_data_df.reset_index(drop=True)
enriched_crop_yield_data_df

PROCESSED 1653/1653

,index,id,location_id,crop_id,sowing_year,sowing_month,harvest_year,harvest_month,_yield,surface_solar_radiation_downwards_mean,...,2m_temperature_mean,2m_temperature_sum,2m_temperature_std,2m_temperature_min,2m_temperature_max,total_precipitation_mean,total_precipitation_sum,total_precipitation_std,total_precipitation_min,total_precipitation_max
0,0,80e4fecf-e240-46b9-b144-950c38b97493,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2400.0,1.492664e+07,...,298.948633,1494.743164,1.290209,297.227305,300.498265,0.005672,0.028358,0.002272,0.002224,0.008091
1,1,d97306b4-053d-4f31-b31c-ad2e1163ddf2,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,2500.0,1.492664e+07,...,298.948633,1494.743164,1.290209,297.227305,300.498265,0.005672,0.028358,0.002272,0.002224,0.008091
2,2,c14f8468-8ca0-454a-9e06-c372a372efd4,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,3093.0,1.492664e+07,...,298.948633,1494.743164,1.290209,297.227305,300.498265,0.005672,0.028358,0.002272,0.002224,0.008091
3,3,5660858a-071e-4cb0-9ad5-14d808b0fb47,3d3b83c1-9dd2-4b5b-a06e-bd1f83a8188c,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,3,1980,7,5678.0,1.492664e+07,...,298.948633,1494.743164,1.290209,297.227305,300.498265,0.005672,0.028358,0.002272,0.002224,0.008091
4,4,c92dc64d-9b4f-4a28-a787-f36913b43cec,423761b0-10b5-4ffa-b56b-ef5df7215e85,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,1980,4,1980,10,4400.0,1.602599e+07,...,288.512288,2019.586018,4.541800,281.377774,293.239102,0.002632,0.018421,0.000924,0.001446,0.003809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648,1648,9c64d4e8-6c5a-4f6b-9c99-196810c096f7,d7636c4f-6399-4275-8fab-4d3f531d898a,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,9,4050.0,2.104846e+07,...,295.281411,1771.688467,4.947208,288.201561,300.141113,0.002919,0.017515,0.001749,0.001644,0.006407
1649,1649,feff9efb-915a-460d-9235-fc4d3b65fc42,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,10895.0,2.072073e+07,...,292.877662,1464.388309,5.774836,284.081490,297.555779,0.002245,0.011225,0.001139,0.000579,0.003528
1650,1650,8c4c0099-7b29-413a-9094-2b81143d84aa,8d3b2bfc-0d0f-4ba2-b2aa-a30fdda829ff,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,4,2017,8,12114.0,2.072073e+07,...,292.877662,1464.388309,5.774836,284.081490,297.555779,0.002245,0.011225,0.001139,0.000579,0.003528
1651,1651,232bd556-4455-41b8-ae2e-92c1d45a758a,e22382f9-1e49-4eb5-acad-9e7d45a9e788,ab7e4a12-5fe5-4701-8cb0-e2f6d084c0f3,2017,5,2017,9,9670.0,1.945064e+07,...,297.234804,1486.174020,2.410495,294.733508,300.307389,0.003349,0.016747,0.002607,0.000901,0.007616
